# Time series correlation

In [5]:
import pyspedas
from pytplot import get_data
from pyspedas.analysis.tvectot import tvectot

import xarray as xr
import polars as pl
import polars.selectors as cs

import hvplot.polars  # noqa

import scipy.stats
import dcor

import astropy.units as u

In [6]:
from datetimerange import DateTimeRange
from sunpy.time import TimeRange
from numpy import timedelta64
from datetime import timedelta

In [7]:
start = "2019-04-06T12:00"
end = "2019-04-07T12:00"

earth_start = "2019-04-09"
earth_end = "2019-04-14"

In [8]:
psp_timerange = TimeRange(start, end)

timerange_earth = TimeRange(earth_start, earth_end)

In [9]:
def validate(timerange):
    if isinstance(timerange, DateTimeRange):
        return [timerange.get_start_time_str(), timerange.get_end_time_str()]
    if isinstance(timerange, TimeRange):
        return [timerange.start.to_string(), timerange.end.to_string()]

## Estimate of arrival time

In [10]:
psp_spi_vars = pyspedas.psp.spi(trange=validate(psp_timerange), time_clip=True)

tvar = tvectot("psp_spi_VEL_RTN_SUN")
psp_ion_speed: xr.DataArray = get_data(tvar, xarray=True)

r_psp = get_data("psp_spi_SUN_DIST", xarray=True)
# r_psp = pl.DataFrame(r_psp.to_dataframe().reset_index())

27-Mar-24 07:53:23: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2021/


Using LEVEL=L3


27-Mar-24 07:53:24: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2021/psp_swp_spi_sf00_l3_mom_20210117_v04.cdf to /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2021/psp_swp_spi_sf00_l3_mom_20210117_v04.cdf
27-Mar-24 07:53:27: Download complete: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2021/psp_swp_spi_sf00_l3_mom_20210117_v04.cdf


TypeError: 'CDFInfo' object is not subscriptable

In [11]:
df_info = pl.DataFrame(
    r_psp.to_dataframe().join(psp_ion_speed.to_dataframe()).reset_index()
)

NameError: name 'r_psp' is not defined

In [ ]:
km2au = u.km.to(u.AU)
r_earth = 1 * u.AU.to(u.km)
v_sw_slow = 250 * u.km / u.s
v_sw_fast = 500 * u.km / u.s

df_info.sort("time").group_by_dynamic("time", every="1h").agg(
    cs.float().mean()
).with_columns(
    distance2sun=pl.col("psp_spi_SUN_DIST"),
    distance2earth=r_earth - pl.col("psp_spi_SUN_DIST"),
).with_columns(
    dt2arrival=pl.duration(
        seconds=pl.col("distance2earth") / pl.col("psp_spi_VEL_RTN_SUN_tot")
    ),
    dt2arrival_slow=pl.duration(seconds=pl.col("distance2earth") / v_sw_slow),
    dt2arrival_fast=pl.duration(seconds=pl.col("distance2earth") / v_sw_fast),
).with_columns(
    time2arrival=pl.col("time") + pl.col("dt2arrival"),
    time2arrival_slow=pl.col("time") + pl.col("dt2arrival_slow"),
    time2arrival_fast=pl.col("time") + pl.col("dt2arrival_fast"),
).with_columns(
    distance2sun=pl.col("distance2sun") * km2au,
    distance2earth=pl.col("distance2earth") * km2au,
)

time,psp_spi_SUN_DIST,psp_spi_VEL_RTN_SUN_tot,distance2sun,distance2earth,dt2arrival,dt2arrival_slow,dt2arrival_fast,time2arrival,time2arrival_slow,time2arrival_fast
datetime[ns],f64,f32,f64,f64,duration[μs],duration[μs],duration[μs],datetime[μs],datetime[μs],datetime[μs]
2019-04-06 12:00:00,2.6162e7,289.611786,0.174885,0.825115,4d 22h 23m 30s,5d 17h 9m 1s,2d 20h 34m 30s,2019-04-11 10:23:30,2019-04-12 05:09:01,2019-04-09 08:34:30
2019-04-06 13:00:00,2.6231e7,273.515167,0.175347,0.824653,5d 5h 17m 20s,5d 17h 4m 25s,2d 20h 32m 12s,2019-04-11 18:17:20,2019-04-12 06:04:25,2019-04-09 09:32:12
2019-04-06 14:00:00,2.6302e7,255.006729,0.175819,0.824181,5d 14h 18m 20s,5d 16h 59m 43s,2d 20h 29m 51s,2019-04-12 04:18:20,2019-04-12 06:59:43,2019-04-09 10:29:51
2019-04-06 15:00:00,2.6374e7,243.91748,0.1763,0.8237,5d 20h 19m 46s,5d 16h 54m 55s,2d 20h 27m 27s,2019-04-12 11:19:46,2019-04-12 07:54:55,2019-04-09 11:27:27
2019-04-06 16:00:00,2.6448e7,244.805847,0.176796,0.823204,5d 19h 44m 9s,5d 16h 49m 58s,2d 20h 24m 59s,2019-04-12 11:44:09,2019-04-12 08:49:58,2019-04-09 12:24:59
2019-04-06 17:00:00,2.6523e7,232.988617,0.177294,0.822706,6d 2h 44m 4s,5d 16h 45m,2d 20h 22m 30s,2019-04-12 19:44:04,2019-04-12 09:45:00,2019-04-09 13:22:30
2019-04-06 18:00:00,2.6599e7,224.188614,0.177805,0.822195,6d 8h 23m 58s,5d 16h 39m 54s,2d 20h 19m 57s,2019-04-13 02:23:58,2019-04-12 10:39:54,2019-04-09 14:19:57
2019-04-06 19:00:00,2.6677e7,222.26709,0.178326,0.821674,6d 9h 37m 11s,5d 16h 34m 42s,2d 20h 17m 21s,2019-04-13 04:37:11,2019-04-12 11:34:42,2019-04-09 15:17:21
2019-04-06 20:00:00,2.6757e7,234.130203,0.178857,0.821143,6d 1h 44m 30s,5d 16h 29m 25s,2d 20h 14m 42s,2019-04-12 21:44:30,2019-04-12 12:29:25,2019-04-09 16:14:42


## Get velocity

In [ ]:
psp_spi_vars = pyspedas.psp.spi(trange=validate(psp_timerange), time_clip=True)
swe_vars = pyspedas.ace.swe(trange=validate(timerange_earth), datatype="h0")

29-Jan-24 14:35:31: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/


Using LEVEL=L3


29-Jan-24 14:35:31: File is current: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/psp_swp_spi_sf00_l3_mom_20190406_v04.cdf
29-Jan-24 14:35:31: File is current: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/psp_swp_spi_sf00_l3_mom_20190407_v04.cdf
29-Jan-24 14:35:32: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/ace/swepam/level_2_cdaweb/swe_h0/2019/
29-Jan-24 14:35:32: File is current: /Users/zijin/data/ace/swepam/level_2_cdaweb/swe_h0/2019/ac_h0_swe_20190409_v11.cdf
29-Jan-24 14:35:33: File is current: /Users/zijin/data/ace/swepam/level_2_cdaweb/swe_h0/2019/ac_h0_swe_20190410_v11.cdf
29-Jan-24 14:35:33: File is current: /Users/zijin/data/ace/swepam/level_2_cdaweb/swe_h0/2019/ac_h0_swe_20190411_v11.cdf
29-Jan-24 14:35:33: File is current: /Users/zijin/data/ace/swepam/level_2_cdaweb/swe_h0/2019/ac_h0_swe_20190412_v11.cdf
29-Jan-24 14:35:33: File is current: /Users/zijin/data/ace/swepam/level_2_cdaweb/swe_h0/2019/ac_h0_swe_20190413_v11.cdf


In [ ]:
tvar = tvectot("psp_spi_VEL_RTN_SUN")
psp_ion_speed: xr.DataArray = get_data(tvar, xarray=True)

ace_ion_speed: xr.DataArray = get_data("Vp", xarray=True)

In [ ]:
def calc_correlation(
    v1: xr.DataArray,
    v2: xr.DataArray,
    v1_timerange: TimeRange,
    v2_timerange: TimeRange,
    cadence: timedelta = timedelta(minutes=10),
    window: timedelta = timedelta(hours=1),
):
    v1 = v1.dropna("time").sel(
        time=slice(v1_timerange.start.to_datetime(), v1_timerange.end.to_datetime())
    )
    v2 = v2.dropna("time").sel(
        time=slice(v2_timerange.start.to_datetime(), v2_timerange.end.to_datetime())
    )
    v1_start = v1_timerange.start.to_datetime()
    v1_end = v1_timerange.end.to_datetime()

    results = []
    for temp_tr in v2_timerange.window(cadence=cadence, window=window):
        v2_temp_start = temp_tr.start.to_datetime()
        v2_temp_end = temp_tr.end.to_datetime()

        v2_temp = v2.sel(time=slice(v2_temp_start, v2_temp_end))
        v2_temp["time"] = v2_temp["time"] - timedelta64(v2_temp_start - v1_start)
        v1_temp = v1.interp_like(v2_temp)

        pearsonr = scipy.stats.pearsonr(v1_temp, v2_temp).statistic
        distance_correlation = dcor.distance_correlation(v1_temp, v2_temp)
        results.append(
            [
                v1_start,
                v1_end,
                v2_temp_start,
                v2_temp_end,
                pearsonr,
                distance_correlation,
            ]
        )

    return pl.DataFrame(
        results,
        schema=[
            "v1_start",
            "v1_end",
            "v2_start",
            "v2_end",
            "Pearson correlation",
            "Distance correlation",
        ],
    )

In [ ]:
window = psp_timerange.end - psp_timerange.start
cadence = timedelta(minutes=10)

df = calc_correlation(
    psp_ion_speed,
    ace_ion_speed,
    psp_timerange,
    timerange_earth,
    cadence=cadence,
    window=window,
)

In [ ]:
df

v1_start,v1_end,v2_start,v2_end,Pearson correlation,Distance correlation
datetime[μs],datetime[μs],datetime[μs],datetime[μs],f64,f64
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:00:00,2019-04-10 00:00:00,0.129733,0.295187
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:10:00,2019-04-10 00:10:00,0.115776,0.290461
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:20:00,2019-04-10 00:20:00,0.125829,0.295799
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:30:00,2019-04-10 00:30:00,0.134056,0.299338
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:40:00,2019-04-10 00:40:00,0.141806,0.309491
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:50:00,2019-04-10 00:50:00,0.137222,0.310114
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 01:00:00,2019-04-10 01:00:00,0.130128,0.307473
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 01:10:00,2019-04-10 01:10:00,0.132468,0.314047
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 01:20:00,2019-04-10 01:20:00,0.137969,0.310688


In [ ]:
df.hvplot("v2_start", ["Pearson correlation", "Distance correlation"])